## Imports

### Python modules

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

## Data Processing

### Dataframe Creation

Read target csv to a Dataframe, ignore all bad lines. Python will tell me which lines are ignored for manual cleanup. Only seems to happen when boards were rebooted - no data loss, easy to fix.

In [2]:
data = pd.read_csv('../../SRAM3/1/half/nvRAM_data_02-06-2021-1720.csv', error_bad_lines=False)
log_length = data.shape[0]

### Neutron Logs

Get first/last log timestamps into datetime objects

In [123]:
exp_tformat = '%Y-%m-%d_%H-%M-%S-%f'
exp_firststamp = datetime.strptime(data.iloc[0]['Timestamp'], exp_tformat)
exp_laststamp = datetime.strptime(data.iloc[log_length-1]['Timestamp'], exp_tformat)

Create neutron log names from datetime objects, import neutron logs ignoring first line

In [148]:
beamlog_fname_tformat = '%Y-%m-%d'
beamlog_firstday = datetime.strftime(exp_firststamp, beamlog_tformat)
beamlog_lastday = datetime.strftime(exp_laststamp, beamlog_tformat)
beamlog_first = pd.read_csv(f"../neutrons/countlog-{beamlog_firstday}.txt", delim_whitespace=True, header=None, skiprows=1)
beamlog_last = pd.read_csv(f"../neutrons/countlog-{beamlog_lastday}.txt", delim_whitespace=True, header=None, skiprows=1)

### Identifying Runs

Create a list of every instance of M,A,WRCHCKBRD. This log line indicates the start of every new experiment run. Should probably make this a user input to this Python so that different experiment setups are easily accounted for.

Create a list of every instance of M,A,INIT. Indicates start of board power cycle.

This method is the easiest, but will label all INIT cycles as a separate experimental run. No loss of data occurs so this is ok.

In [4]:
wrchckbrd_locs = data.loc[(data['Board'] == 'M') & (data['Mikroe_socket'] == 'A') & (data['Status'] == 'WRCHCKBRD')].index.tolist()
wrchckbrd_index = [(i, 'WRCHCKBRD') for i in wrchckbrd_locs]
init_locs = data.loc[(data['Board'] == 'M') & (data['Mikroe_socket'] == 'A') & (data['Status'] == 'INIT')].index.tolist()
init_index = [(i, 'INIT') for i in init_locs]

Combine both lists, and append new list with final log line number.

In [5]:
run_index = wrchckbrd_index + init_index
run_index = sorted(run_index)
run_index.append((log_length, 'END'))

For each element h in c, if the status c[h][1] is WRCHCKBRD, label everything between its index c[h][0] and the next index c[h+1][0] in c with the same run ID. If the status is INIT, no labels. This ensures that only actual test sequences are associated as runs.

In [6]:
run_id = 1
run_col = ['']*log_length
for h in range(len(run_index)-1):
    if run_index[h][1] == 'WRCHCKBRD':
        start = run_index[h][0]
        end = run_index[h+1][0]
        run_col[start:end] = [run_id]*(end-start)
        run_id+=1

In [7]:
data.insert(loc=0, column='Run_ID', value=run_col)

### Hot and Cold Runs

Format datetime object into neutron log timestamps

In [149]:
beamlog_stamp_tformat = '%d/%m/%Y %H:%M:%S'
beamlog_firststamp = datetime.strftime(exp_firststamp, beamlog_stamp_tformat)
beamlog_laststamp = datetime.strftime(exp_laststamp, beamlog_stamp_tformat)


Remake neutron log column names

In [150]:
beamlog_new_colnames = ["Date", "HMS_time", "Millisecs", "Count1", "Count2", "Count3", "Count4", "unknown", "Beam_current"]
beamlog_first.columns = beamlog_new_colnames
beamlog_last.columns = beamlog_new_colnames

### How many bits tend to get flipped?

Get occurrences of each type of error

In [22]:
num_see = data.loc[(data['Status'] == 'SDC')].index
see = data['SDC_val']
see_types_hex = see.value_counts().index.tolist()
see_freqs = see.value_counts().tolist()
see_bins = list(zip(see_types_hex, see_freqs))

### Where do errors tend to happen?

In [24]:
see_loc = data['SDC_loc']
see_locs_hex = see_loc.value_counts().index.tolist()
see_loc_freqs = see_loc.value_counts().tolist()
see_locs_bins = list(zip(see_locs_hex, see_loc_freqs))

Use this for Hamming distance between errors and golden value:

In [83]:
xortest1 = 169
xortest2 = 170

bin(xortest1 ^ xortest2).count('1')

2